In [0]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 2.9MB/s 


In [0]:
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
import random
import matplotlib
from matplotlib import pyplot as plt
import heapq
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link="https://drive.google.com/open?id=1f38gwgHP-Wosesj3u2XF9vuqyWDy_yan"
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Hotel_Reviews.csv')  
df = pd.read_csv('Hotel_Reviews.csv')

1f38gwgHP-Wosesj3u2XF9vuqyWDy_yan


In [0]:
df.head()
df.shape
print(df.Hotel_Name.nunique(), 'hotels in the dataset')


1492 hotels in the dataset


In [0]:
import re, nltk
nltk.download()
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

In [0]:
wordnet_lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer
stop_words = set(stopwords.words('english'))


In [0]:
df_com = df[['Hotel_Name','Negative_Review','Review_Total_Negative_Word_Counts','Positive_Review','Review_Total_Positive_Word_Counts','Total_Number_of_Reviews']]
grouped = df_com.groupby('Hotel_Name')
services=['bar','swimingpool',"swimming pool",'ac','air conditioner','air conditioning','parking','reception','wifi','wi fi','service',"ammenities","room","executive lounge"]
location=['city','surroundings','location',"place","traffic","noise","main road",""]
cleanliness=['tidy','clean','area',"washroom","room","bathroom","floor","bed","light","bathtub","bath tub","shower","tap","table","cupboard","bedsheet","pillow","mattress"]


In [26]:
def rating(temp,temp1):
  servicerating=0
  locationrating=0
  cleanlinessrating=0
  for k in temp:
      if(k in services):
        servicerating+=1
      elif(k in location):
        locationrating+=1
      elif(k in cleanliness):
        cleanlinessrating+=1
  for k in temp1:
      if(k in services):
        servicerating+=1
      elif(k in location):
        locationrating+=1
      elif(k in cleanliness):
        cleanlinessrating+=1
  return servicerating,locationrating,cleanlinessrating        

def rate(s):
  if(s>=0 and s<=20):
    r=1
  elif(s>20 and s<=40):
    r=2
  elif(s>40 and s<=60):
    r=3
  elif(s>60 and s<=80):
    r=4
  else:
    r=5
  return r


def words(list1):
  neg=[]
  for i in list1:
    
    letters = re.sub('[^a-zA-Z]',' ',i)
    tokens = nltk.word_tokenize(letters)
    lowercase = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stop_words, lowercase))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result] 
    neg.append(' '.join(lemmas))
    
  cv = CountVectorizer(analyzer = 'word',stop_words = 'english',max_features = 30,ngram_range=(1,1))
  most_negative_words = cv.fit_transform(neg)
  temp_counts = most_negative_words.sum(axis=0)
  temp_words = cv.vocabulary_
  
  cv1 = CountVectorizer(analyzer = 'word',stop_words = 'english',max_features = 30,ngram_range=(2,2))
  most_negative_words1 = cv1.fit_transform(neg)
  temp_counts1 = most_negative_words1.sum(axis=0)
  temp_words1 = cv1.vocabulary_
  

  sn,ln,cn=rating(temp_words,temp_words1)
  
  return sn,ln,cn

def findingHotel(hotelname):
  for name,group in grouped:
    if(name==hotelname):
      servicerating=0
      locationrating=0
      cleanlinessrating=0
      sn,ln,cn=words(group['Negative_Review'])
      sp,lp,cp=words(group['Positive_Review'])
  
      if((sn+sp)!=0):
        sr=(sp/(sn+sp))*100
        frs=rate(sr)
      else:
        frs= 0
  
      if((cn+cp)!=0):
        cr=(cp/(cn+cp))*100
        frc= rate(cr)
      else:
        frc= 0
  
      if((ln+lp)!=0):
        lr=(lp/(ln+lp))*100
        frl= rate(lr)
      else:
        frl= 0
      return frs,frc,frl
  
  
print("\n\n\t\tChoose Your Ideal Hotel Based on Reviews of 100's of customers\n\n")
print("We have a total of " ,df.Hotel_Name.nunique() ," for you to choose From")
indexes=random.sample(list(df['Hotel_Name'].unique()),6)
j=1
for i in indexes:
  print(j,")   ",i)
  j=j+1
n=int(input("Enter hotel number to search for its rating: \n"))
sf,cf,lf=findingHotel(indexes[n-1])
print("\n\n\n\t\t",indexes[n-1])
print("\n\n---------------------------------------\n\n")
print("Service Rating : ",sf,"/5")
print("Cleanliness Rating : ",cf,"/5")
print("Location Rating : ",lf,"/5")
print("\n\n---------------------------------------\n\n")





		Choose Your Ideal Hotel Based on Reviews of 100's of customers


We have a total of  1492  for you to choose From
1 )    Hotel Astra Opera Astotel
2 )    Hotel Le Mareuil
3 )    Colombia
4 )    Medinaceli
5 )    Eurostars Ramblas
6 )    St George Hotel
Enter hotel number to search for its rating: 
6



		 St George Hotel


---------------------------------------


Service Rating :  2 /5
Cleanliness Rating :  2 /5
Location Rating :  3 /5


---------------------------------------


